In [ ]:
!pip install deepface

In [16]:
from deepface import DeepFace
import cv2
import numpy as np
import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Flatten

In [17]:
import zipfile
import os

# Unzip the uploaded faces.zip file
with zipfile.ZipFile("faces.zip", 'r') as zip_ref:
    zip_ref.extractall("faces")

In [18]:
# List image files and read labels
faces_dir = "/content/faces"
labels_path = "/content/labels.txt"

image_files = sorted(glob.glob(os.path.join(faces_dir, "*.jpg")))

# Load labels from labels.txt
labels = []
with open(labels_path, 'r') as file:
    for line in file:
        parts = line.strip().split()
        label = int(parts[0])
        labels.append(label)

# Split images and labels into training and testing sets
train_files, test_files, y_train, y_test = train_test_split(
    image_files, labels, test_size=0.2, random_state=42, shuffle=True
)

train_files, val_files, y_train, y_val = train_test_split(
    train_files, y_train, test_size=0.1, random_state=42, shuffle=True
)

In [19]:
# Function to generate embeddings
def generate_embeddings(image_paths, labels):
    embeddings = []

    # Process each image and extract embeddings
    for path in image_paths:
        # Read and preprocess image
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        img = cv2.resize(img, (160, 160))  # Resize to input size
        img = img.astype("float32") / 255.0  # Normalize image

        # Extract embeddings using DeepFace
        result = DeepFace.represent(img_path=path, model_name="VGG-Face", enforce_detection=False)
        if result:
            embeddings.append(np.array(result[0]["embedding"]))
        else:
            print(f"Face not detected in {path}")

    return np.array(embeddings)


# Generate embeddings
X_train_embeddings = generate_embeddings(train_files, y_train)
X_val_embeddings = generate_embeddings(val_files, y_val)
X_test_embeddings = generate_embeddings(test_files, y_test)

print(f"Train embeddings shape: {X_train_embeddings.shape}")
print(f"Validation embeddings shape: {X_val_embeddings.shape}")
print(f"Test embeddings shape: {X_test_embeddings.shape}")

Train embeddings shape: (2880, 4096)
Validation embeddings shape: (320, 4096)
Test embeddings shape: (800, 4096)


In [20]:
# Convert labels to NumPy arrays
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

In [50]:
# Build a neural network classifier
classifier = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4096,)),  # Input layer for VGG-Face embeddings
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(1, activation='sigmoid')  # Binary output for smile/non-smile
])

learning_rate = 1e-5 * 5
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model with the custom learning rate
classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = classifier.fit(X_train_embeddings, y_train,
                         epochs=10,
                         batch_size=32,
                         validation_data=(X_val_embeddings, y_val))

# Evaluate the model
test_loss, test_accuracy = classifier.evaluate(X_test_embeddings, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Epoch 1/10
90/90 [==============================] - 10s 101ms/step - loss: 0.6801 - accuracy: 0.5684 - val_loss: 0.6425 - val_accuracy: 0.6844
Epoch 2/10
90/90 [==============================] - 8s 85ms/step - loss: 0.5425 - accuracy: 0.7976 - val_loss: 0.3746 - val_accuracy: 0.8844
Epoch 3/10
90/90 [==============================] - 9s 102ms/step - loss: 0.3090 - accuracy: 0.8851 - val_loss: 0.2432 - val_accuracy: 0.8969
Epoch 4/10
90/90 [==============================] - 8s 88ms/step - loss: 0.2213 - accuracy: 0.9174 - val_loss: 0.2066 - val_accuracy: 0.9375
Epoch 5/10
90/90 [==============================] - 9s 100ms/step - loss: 0.1811 - accuracy: 0.9326 - val_loss: 0.1962 - val_accuracy: 0.9156
Epoch 6/10
90/90 [==============================] - 8s 91ms/step - loss: 0.1432 - accuracy: 0.9538 - val_loss: 0.1948 - val_accuracy: 0.9250
Epoch 7/10
90/90 [==============================] - 9s 101ms/step - loss: 0.1155 - accuracy: 0.9622 - val_loss: 0.1934 - val_accuracy: 0.9250
Epoch 8/

In [ ]:
# Save the trained model
classifier.save('/content/smile_non_smile_classifier.h5')